In [136]:
#Imports and setting global variables
import math
import json
import pandas as pd
import numpy as np
import os
import requests
from datetime import datetime
import matplotlib.pyplot as plt
userOneData = pd.read_json('/work/proveChallenge/user1.json')
sampleTestData = pd.read_json('/work/proveChallenge/sampleTest/sample_test.json')
correctSequence = "Be Authentic. Be Yourself. Be Typing."

In [137]:
#Create the dataset and downloading all training data from the system
userData = pd.read_json('user1.json')
userCount = 1
while isinstance(userData['next'][0],str) is True:
    userCount += 1
    url = "https://challenges.unify.id/v1/mle/" + userData['next'][0]
    r = requests.get(url, allow_redirects = True)
    fileName = 'user' + str(userCount) + ".json"
    open(fileName, 'wb').write(r.content)
    userData = pd.read_json(fileName)

In [113]:
def calculateError(userJsonData):
    userData = pd.read_json(userJsonData)
    errorArr = []
    for i in range(len(userData['user_data'])):
        errorCounter = 0
        for j in range(len(userData['user_data'][i])):
            if(userData['user_data'][i][j]['character'] == '[backspace]'):
                errorCounter += 1
        errorArr.append(errorCounter/len(userData['user_data'][i]))
    totalError = 0
    for i in range(len(errorArr)):
        totalError += errorArr[i]

    return totalError/len(errorArr)

def calculateSequenceError(sequence):
    errorCounter = 0
    for i in range(len(sequence)):
        if(sequence[i]['character'] == '[backspace]'):
            errorCounter += 1
    return errorCounter/len(sequence)
print(calculateError('user1.json'))

def createStr(attempt, correctSequence):
    currStr = ""
    for char in attempt:
        if(char['character'] == '[backspace]'):
            currStr = currStr[:-1]
        else:
            currStr += char['character']
    return currStr
        
def numCorrectSequences(userJsonData, correctSequence):
    typedStringArr = []
    correctStringCounter = 0
    userData = pd.read_json(userJsonData)
    for charSequence in userData['user_data']:
        currStr = ""
        for char in charSequence:
            if(char['character'] == '[backspace]'):
                currStr = currStr[:-1]
            else:
                currStr += char['character']
        typedStringArr.append(currStr)
        if(currStr == correctSequence):
            correctStringCounter += 1  
    return correctStringCounter,typedStringArr

numCorrectStrings = numCorrectSequences('user1.json', correctSequence)

0.035808255358961386


In [114]:
def findCorrectIndices(userJsonData, correctSequence):
    correctIndexArr = []
    userData = pd.read_json(userJsonData)
    for i in range(len(userData['user_data'])):
        currStr = ""
        for char in userData['user_data'][i]:
            if(char['character'] == '[backspace]'):
                currStr = currStr[:-1]
            else:
                currStr += char['character']
        if(currStr == correctSequence):
            correctIndexArr.append(i)
    return correctIndexArr
correctIndexArr = findCorrectIndices('user1.json', correctSequence)

In [115]:
#Determine users that are qualifying for enrollment in production
qualifyingUserArr = []
nonQualifyingUserArr= []
directory = '/work'
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        numCorrectStrings = numCorrectSequences(filename,correctSequence)[0]
        if(numCorrectStrings >= 300):
            qualifyingUserArr.append(filename)
        else:
            nonQualifyingUserArr.append(filename)

print(qualifyingUserArr) #users that could be enrolled in production
print(nonQualifyingUserArr) #users that can't be enrolled in production

['user31.json', 'user18.json', 'user29.json', 'user35.json', 'user10.json', 'user21.json', 'user39.json', 'user25.json', 'user8.json', 'user11.json', 'user20.json', 'user38.json', 'user24.json', 'user9.json', 'user30.json', 'user28.json', 'user5.json', 'user34.json', 'user1.json', 'user13.json', 'user22.json', 'user40.json', 'user26.json', 'user17.json', 'user32.json', 'user7.json', 'user36.json', 'user3.json', 'user33.json', 'user6.json', 'user2.json', 'user12.json', 'user23.json', 'user27.json', 'user16.json']
['user4.json', 'user14.json', 'user15.json', 'user19.json', 'user37.json']


In [116]:
def calculateTimeDifference(t1, t2):
    fmt = '%Y-%m-%dT%H:%M:%S.%f'
    tstamp1 = datetime.strptime(t1, fmt)
    tstamp2 = datetime.strptime(t2, fmt)
    if tstamp1 > tstamp2:
        td = tstamp1 - tstamp2
    else:
        td = tstamp2 - tstamp1
    return td.microseconds

def calculateTimeAvg(sequence):
    timeSum = 0
    for i in range(len(sequence) - 1):
        timeSum += calculateTimeDifference(sequence[i]['typed_at'], sequence[i+1]['typed_at'])
    return timeSum/len(sequence) 

def calculateTimeDifferencesArr(userJsonData):
    correctIndexArr = findCorrectIndices(userJsonData, correctSequence)
    overallTimeDifArr = []
    userData = pd.read_json(userJsonData)
    for i in correctIndexArr:
        sequenceTimeDif = []
        for j in range(len(userData['user_data'][i]) - 1):
            td = calculateTimeDifference(userData['user_data'][i][j]['typed_at'], userData['user_data'][i][j+1]['typed_at'])
            sequenceTimeDif.append(td)
        overallTimeDifArr.append(sequenceTimeDif)
    return overallTimeDifArr

def avgTimeDifference(userJsonData):
    userData = pd.read_json(userJsonData)
    timeArr = calculateTimeDifferencesArr(userJsonData)
    avgTimeArr = []
    timeSum = 0
    lenSum = 0
    for i in range(len(timeArr)):
        lenSum += len(timeArr[i])
        for j in range(len(timeArr[i])):
            timeSum += timeArr[i][j]
        #timeSum = sum(z[i])
    return timeSum/lenSum
#print(userData['user_data'][0][0]['typed_at'])
#z = calculateTimeDifferencesArr('user1.json')



In [117]:
overallAverageTimeArr = []
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        averageTime = avgTimeDifference(filename)
        overallAverageTimeArr.append([averageTime,filename])
        #print([averageTime, filename])
print(overallAverageTimeArr)

[[30897.584375574766, 'user31.json'], [17529.916418355184, 'user18.json'], [28870.451104508462, 'user29.json'], [4419.174680306905, 'user4.json'], [35126.3481733665, 'user35.json'], [9328.861072902338, 'user10.json'], [20587.506492842534, 'user21.json'], [39074.8740836063, 'user39.json'], [24605.036920714683, 'user25.json'], [7878.958452473789, 'user8.json'], [13430.5158008658, 'user14.json'], [10487.669098712446, 'user11.json'], [19564.29101651265, 'user20.json'], [37988.70306172039, 'user38.json'], [23608.381692287305, 'user24.json'], [9139.312947358883, 'user9.json'], [14683.68936210848, 'user15.json'], [29845.641366160435, 'user30.json'], [18608.256449012493, 'user19.json'], [27807.023966446974, 'user28.json'], [8211.541039895357, 'user5.json'], [33889.80978585162, 'user34.json'], [7596.237415992319, 'user1.json'], [13119.108055862396, 'user13.json'], [21786.025907765703, 'user22.json'], [40089.00407955125, 'user40.json'], [25741.927442748092, 'user26.json'], [16706.053511942733, '

In [118]:
overallErrorArr = []
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        averageError = calculateError(filename)
        overallErrorArr.append([averageError, filename])
print(overallErrorArr)

[[0.03282424903504735, 'user31.json'], [0.04236737502636323, 'user18.json'], [0.03771850884205978, 'user29.json'], [0.0269061023675613, 'user4.json'], [0.03774625313023953, 'user35.json'], [0.031735249685318295, 'user10.json'], [0.035858488510808306, 'user21.json'], [0.028878100589092823, 'user39.json'], [0.03302165597684601, 'user25.json'], [0.030593757833612994, 'user8.json'], [0.027654892479171293, 'user14.json'], [0.02762635073289762, 'user11.json'], [0.03958999191283364, 'user20.json'], [0.03684984103616817, 'user38.json'], [0.034368432536339394, 'user24.json'], [0.035074363702354325, 'user9.json'], [0.030793397309349692, 'user15.json'], [0.026747336693446675, 'user30.json'], [0.03282833653624611, 'user19.json'], [0.028734288817729093, 'user28.json'], [0.026916848436777083, 'user5.json'], [0.030938639760442258, 'user34.json'], [0.035808255358961386, 'user1.json'], [0.022593579855760316, 'user13.json'], [0.024396420064155058, 'user22.json'], [0.031948984014131696, 'user40.json'], [

In [135]:
featureVectorArr = []
for i in range(len(overallErrorArr)):
    featureVectorArr.append([overallErrorArr[i][0],overallAverageTimeArr[i][0],overallErrorArr[i][1]])


In [132]:
#predictions using both error rate and average latency

test = pd.read_json('/work/sampleTest/sample_test.json')
attempts = test['attempts']
testAverageTimeArr = []
for i in range(len(attempts)):
    testAverageTimeArr.append([calculateTimeAvg(attempts[i]),i])

testAverageErrorArr = []
for i in range(len(attempts)):
    testAverageErrorArr.append([calculateSequenceError(attempts[i]),i])
#print(testAverageErrorArr)


testVectorArr = []
for i in range(len(testAverageErrorArr)):
    testVectorArr.append([testAverageErrorArr[i][0], testAverageTimeArr[i][0]])
#print(testVectorArr)

minSquaredVal = math.inf
minUser = None
predictionArr = []
for i in range(len(testVectorArr)):
    for j in range(len(featureVectorArr)):
        squaredVal = (testVectorArr[i][0] - featureVectorArr[j][0])**2 + (testVectorArr[i][1] - featureVectorArr[j][1])**2
        if squaredVal < minSquaredVal:
            minUser = featureVectorArr[j][2]
            minSquaredVal = squaredVal
    predictionArr.append(minUser)
print(predictionArr)




['user28.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user25.json', 'user25.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user33.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.json', 'user31.j

In [134]:
#prediction using only average latency
predictionArr = []

for i in range(len(testAverageTimeArr)):
    minUserTime = math.inf
    minErrorDif = math.inf
    minUser = None
    for j in range(len(overallAverageTimeArr)):
        if(abs(overallAverageTimeArr[j][0] - testAverageTimeArr[i][0]) < minUserTime):
            minUserTime = abs(overallAverageTimeArr[j][0] - testAverageTimeArr[i][0])
        if(abs(overallErrorArr[j][0] - testAverageErrorArr[i][0]) < minErrorDif):
            minErrorDif = abs(overallErrorArr[j][0] - testAverageErrorArr[i][0])
        


for i in range(len(testAverageTimeArr)):
    minUserTime = math.inf
    minUser = None
    for j in range(len(overallAverageTimeArr)):
        if(abs(overallAverageTimeArr[j][0] - testAverageTimeArr[i][0]) < minUserTime):
            minUserTime = abs(overallAverageTimeArr[j][0] - testAverageTimeArr[i][0])
            minUser = overallAverageTimeArr[j][1]
    predictionArr.append(minUser)

print(predictionArr)

['user28.json', 'user31.json', 'user39.json', 'user23.json', 'user12.json', 'user13.json', 'user15.json', 'user24.json', 'user9.json', 'user32.json', 'user9.json', 'user32.json', 'user28.json', 'user18.json', 'user15.json', 'user25.json', 'user7.json', 'user18.json', 'user22.json', 'user39.json', 'user26.json', 'user25.json', 'user14.json', 'user22.json', 'user11.json', 'user22.json', 'user37.json', 'user32.json', 'user12.json', 'user25.json', 'user4.json', 'user33.json', 'user6.json', 'user22.json', 'user28.json', 'user37.json', 'user39.json', 'user36.json', 'user35.json', 'user28.json', 'user35.json', 'user9.json', 'user11.json', 'user15.json', 'user31.json', 'user17.json', 'user11.json', 'user31.json', 'user4.json', 'user23.json', 'user38.json', 'user13.json', 'user21.json', 'user35.json', 'user7.json', 'user5.json', 'user28.json', 'user26.json', 'user31.json', 'user9.json', 'user5.json', 'user20.json', 'user11.json', 'user22.json', 'user22.json', 'user25.json', 'user5.json', 'user1

Additional Questions
* If you had one additional day, what would you change or improve to your submission?
If I had an additional day, I would train a neural network with additional features outside of just the latency time, which is the difference in time between the keystrokes, and the error rate, which I calculated to be the number of backspaces entered into the keystrokes divided by the total number of keystrokes. I would try to come up with additional features and try to increase the dataset as well in order to get more input data possibly. 
* How would you modify your solution if the number of users was 1,000 times larger?
If I had a number of users that was 1000 times larger, I wouldn't be looping through each of the users and would create a neural network. Currently, I am doing a binary classification model for each pair of test value and users, which takes a lot of time. In the case of there being 1,000 times as many users, the time necessary to undergo such a process is not feasible. With a neural network, we would be able to train the network once and then run the input test data each time instead of going through multiple binary classifications each time. 
* What insights and takeaways do you have on the distribution of user performance?
After graphing and checking the latency for each of the users, I saw that the latency varied by quite a bit. After calculating the average latency for each of the 40 users, the variation in the average latency was very high. There were some users where the average latency was around 7000 milliseconds and some users where the average latency was around 30000 milliseconds. This is an average change of almost 4 times, which is quite a large change. 
* What aspect(s) would you change about this challenge?
I would change the dataset to have more information about the keystrokes outside of just the latency. After reading some research papers, I saw that a lot of models took into account other factors outside of just latency such as the time between starting to press down one key and raising that same key. These different features would have increased my accuracy and given more datapoints and input data in order to potentially create a neural network over the relatively simple model I currently have. 
* What aspect(s) did you enjoy about this challenge?
I thought it was a very interesting challenge. I thought it was difficult to go from start to finish in terms of creating a model for using this biometric keystroke analysis but I learned a lot. I was able to create the dataset, come up with different features, do research to come up with these features, perform feature generation and finally come up and choose a model. Although it was difficult to do so in the given time, it was a fruitful experience and taught me a lot. I'm looking forward to continuing to work on this project and improve my accuracy. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=79e6e01b-7569-4af6-9074-989d11ef3b00' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>